## Trabalho de conclusão do curso de Especialização em Ciência de Dados da Facens
## Faculdade de Engenharia de Sorocaba

**Alunos:** 
<p>
<p>Dani Shizue Sakamoto
<p>Mariana de Castro Pires Tavares Albuquerque
<p>Tan Kin Wah

In [ ]:
## 3.1.Execução do modelo gerado

Essa é a parte mais legal, onde consolida-se tudo que trabalhamos nos topicos anterior e visualiza isso na prática.

- Execução do modelo original genero e idade 


- Video e imagem estática (foto).

# INPUT WEBCAM

In [ ]:
# Import required modules
import cv2 as cv
import math
import time
import argparse

def getFaceBox(net, frame, conf_threshold=0.7):
    frameOpencvDnn = frame.copy()
    frameHeight = frameOpencvDnn.shape[0]
    frameWidth = frameOpencvDnn.shape[1]
    blob = cv.dnn.blobFromImage(frameOpencvDnn, 1.0, (300, 300), [104, 117, 123], True, False)

    net.setInput(blob)
    detections = net.forward()
    bboxes = []
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > conf_threshold:
            x1 = int(detections[0, 0, i, 3] * frameWidth)
            y1 = int(detections[0, 0, i, 4] * frameHeight)
            x2 = int(detections[0, 0, i, 5] * frameWidth)
            y2 = int(detections[0, 0, i, 6] * frameHeight)
            bboxes.append([x1, y1, x2, y2])
            cv.rectangle(frameOpencvDnn, (x1, y1), (x2, y2), (0, 255, 0), int(round(frameHeight/150)), 8)
    return frameOpencvDnn, bboxes



faceProto = r"C:\Users\KinWah\MASK_TCC\AgeGender\opencv_face_detector.pbtxt"
faceModel = r"C:\Users\KinWah\MASK_TCC\AgeGender\opencv_face_detector_uint8.pb"

ageProto = r"C:\Users\KinWah\MASK_TCC\AgeGender\age_deploy.prototxt"
ageModel = r"C:\Users\KinWah\MASK_TCC\AgeGender\age_net.caffemodel"

genderProto = r"C:\Users\KinWah\MASK_TCC\AgeGender\gender_deploy.prototxt"
genderModel = r"C:\Users\KinWah\MASK_TCC\AgeGender\gender_net.caffemodel"

MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
ageList = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
genderList = ['Male', 'Female']

# Load network
ageNet = cv.dnn.readNet(ageModel, ageProto)
genderNet = cv.dnn.readNet(genderModel, genderProto)
faceNet = cv.dnn.readNet(faceModel, faceProto)

# Open a video file or an image file or a camera stream
cap = cv.VideoCapture(0)
padding = 20
while cv.waitKey(1) < 0:
    # Read frame
    t = time.time()
    hasFrame, frame = cap.read()
    if not hasFrame:
        cv.waitKey()
        break

    frameFace, bboxes = getFaceBox(faceNet, frame)
    if not bboxes:
        print("No face Detected, Checking next frame")
        continue

    for bbox in bboxes:
        # print(bbox)
        face = frame[max(0,bbox[1]-padding):min(bbox[3]+padding,frame.shape[0]-1),max(0,bbox[0]-padding):min(bbox[2]+padding, frame.shape[1]-1)]

        blob = cv.dnn.blobFromImage(face, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
        genderNet.setInput(blob)
        genderPreds = genderNet.forward()
        gender = genderList[genderPreds[0].argmax()]
        # print("Gender Output : {}".format(genderPreds))
        print("Gender : {}, conf = {:.3f}".format(gender, genderPreds[0].max()))

        ageNet.setInput(blob)
        agePreds = ageNet.forward()
        age = ageList[agePreds[0].argmax()]
        print("Age Output : {}".format(agePreds))
        print("Age : {}, conf = {:.3f}".format(age, agePreds[0].max()))

        label = "{},{}".format(gender, age)
        cv.putText(frameFace, label, (bbox[0], bbox[1]-10), cv.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2, cv.LINE_AA)
        cv.imshow("Age Gender Demo", frameFace)
        # cv.imwrite("age-gender-out-{}".format(args.input),frameFace)
        
    print("time : {:.3f}".format(time.time() - t))
    key = cv.waitKey(1)
    if key == ord('q'):
        break

cap.release()
cv.destroyAllWindows()

Gender : Female, conf = 0.950
Age Output : [[2.1335389e-02 8.9351666e-01 8.1158362e-02 6.8390602e-04 2.9188253e-03
  2.0585276e-04 6.0574006e-05 1.2025505e-04]]
Age : (4-6), conf = 0.894
time : 0.419
Gender : Male, conf = 1.000
Age Output : [[2.97902534e-06 7.96310269e-05 8.17782640e-01 1.80012733e-01
  2.05328059e-03 4.68476792e-05 1.28828415e-05 9.02751344e-06]]
Age : (8-12), conf = 0.818
time : 0.253
Gender : Male, conf = 1.000
Age Output : [[3.5109651e-06 8.9634479e-05 8.2243353e-01 1.7311731e-01 4.2315815e-03
  7.3428178e-05 3.3412925e-05 1.7591830e-05]]
Age : (8-12), conf = 0.822
time : 0.152
Gender : Male, conf = 1.000
Age Output : [[2.68397434e-06 6.47387715e-05 8.75544786e-01 1.20697334e-01
  3.58582917e-03 6.25588727e-05 2.76498886e-05 1.43706993e-05]]
Age : (8-12), conf = 0.876
time : 0.175
Gender : Male, conf = 1.000
Age Output : [[5.1647612e-06 1.1847026e-04 7.3593050e-01 2.5883573e-01 4.9289525e-03
  1.1712677e-04 4.1913176e-05 2.2221806e-05]]
Age : (8-12), conf = 0.736
t

# Input foto

In [ ]:
# Import required modules
import cv2 as cv
import math
import time
import argparse

def getFaceBox(net, frame, conf_threshold=0.7):
    frameOpencvDnn = frame.copy()
    frameHeight = frameOpencvDnn.shape[0]
    frameWidth = frameOpencvDnn.shape[1]
    blob = cv.dnn.blobFromImage(frameOpencvDnn, 1.0, (300, 300), [104, 117, 123], True, False)

    net.setInput(blob)
    detections = net.forward()
    bboxes = []
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > conf_threshold:
            x1 = int(detections[0, 0, i, 3] * frameWidth)
            y1 = int(detections[0, 0, i, 4] * frameHeight)
            x2 = int(detections[0, 0, i, 5] * frameWidth)
            y2 = int(detections[0, 0, i, 6] * frameHeight)
            bboxes.append([x1, y1, x2, y2])
            cv.rectangle(frameOpencvDnn, (x1, y1), (x2, y2), (0, 255, 0), int(round(frameHeight/150)), 8)
    return frameOpencvDnn, bboxes



faceProto = r"C:\Users\KinWah\MASK_TCC\AgeGender\opencv_face_detector.pbtxt"
faceModel = r"C:\Users\KinWah\MASK_TCC\AgeGender\opencv_face_detector_uint8.pb"

ageProto = r"C:\Users\KinWah\MASK_TCC\AgeGender\age_deploy.prototxt"
ageModel = r"C:\Users\KinWah\MASK_TCC\AgeGender\age_net.caffemodel"

genderProto = r"C:\Users\KinWah\MASK_TCC\AgeGender\gender_deploy.prototxt"
genderModel = r"C:\Users\KinWah\MASK_TCC\AgeGender\gender_net.caffemodel"

MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
ageList = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
genderList = ['Male', 'Female']

# Load network
ageNet = cv.dnn.readNet(ageModel, ageProto)
genderNet = cv.dnn.readNet(genderModel, genderProto)
faceNet = cv.dnn.readNet(faceModel, faceProto)

# Open a video file or an image file or a camera stream
cap = cv.VideoCapture(r"C:\Users\KinWah\MASK_TCC\AgeGender\images\16.jpg")
padding = 20
while cv.waitKey(1) < 0:
    # Read frame
    t = time.time()
    hasFrame, frame = cap.read()
    if not hasFrame:
        cv.waitKey()
        break

    frameFace, bboxes = getFaceBox(faceNet, frame)
    if not bboxes:
        print("No face Detected, Checking next frame")
        continue

    for bbox in bboxes:
        # print(bbox)
        face = frame[max(0,bbox[1]-padding):min(bbox[3]+padding,frame.shape[0]-1),max(0,bbox[0]-padding):min(bbox[2]+padding, frame.shape[1]-1)]

        blob = cv.dnn.blobFromImage(face, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
        genderNet.setInput(blob)
        genderPreds = genderNet.forward()
        gender = genderList[genderPreds[0].argmax()]
        # print("Gender Output : {}".format(genderPreds))
        print("Gender : {}, conf = {:.3f}".format(gender, genderPreds[0].max()))

        ageNet.setInput(blob)
        agePreds = ageNet.forward()
        age = ageList[agePreds[0].argmax()]
        print("Age Output : {}".format(agePreds))
        print("Age : {}, conf = {:.3f}".format(age, agePreds[0].max()))

        label = "{},{}".format(gender, age)
        cv.putText(frameFace, label, (bbox[0], bbox[1]-10), cv.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2, cv.LINE_AA)
        cv.imshow("Age Gender Demo", frameFace)
        # cv.imwrite("age-gender-out-{}".format(args.input),frameFace)
        
    print("time : {:.3f}".format(time.time() - t))
    key = cv.waitKey(1)
    if key == ord('q'):
        break

cap.release()
cv.destroyAllWindows()

Gender : Male, conf = 1.000
Age Output : [[3.4487228e-05 1.2291026e-04 2.3906452e-03 4.3789260e-03 9.8932183e-01
  3.2281633e-03 3.5561231e-04 1.6742549e-04]]
Age : (25-32), conf = 0.989
time : 0.190


# CRIANCA - ADOLESCENTE - ADULTO

In [ ]:
# Import required modules
import cv2 as cv
import math
import time
import argparse

def getFaceBox(net, frame, conf_threshold=0.7):
    frameOpencvDnn = frame.copy()
    frameHeight = frameOpencvDnn.shape[0]
    frameWidth = frameOpencvDnn.shape[1]
    blob = cv.dnn.blobFromImage(frameOpencvDnn, 1.0, (300, 300), [104, 117, 123], True, False)

    net.setInput(blob)
    detections = net.forward()
    bboxes = []
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > conf_threshold:
            x1 = int(detections[0, 0, i, 3] * frameWidth)
            y1 = int(detections[0, 0, i, 4] * frameHeight)
            x2 = int(detections[0, 0, i, 5] * frameWidth)
            y2 = int(detections[0, 0, i, 6] * frameHeight)
            bboxes.append([x1, y1, x2, y2])
            cv.rectangle(frameOpencvDnn, (x1, y1), (x2, y2), (0, 255, 0), int(round(frameHeight/150)), 8)
    return frameOpencvDnn, bboxes



faceProto = r"C:\Users\KinWah\MASK_TCC\AgeGender\opencv_face_detector.pbtxt"
faceModel = r"C:\Users\KinWah\MASK_TCC\AgeGender\opencv_face_detector_uint8.pb"

ageProto = r"C:\Users\KinWah\MASK_TCC\AgeGender\age_deploy.prototxt"
ageModel = r"C:\Users\KinWah\MASK_TCC\AgeGender\age_net.caffemodel"

genderProto = r"C:\Users\KinWah\MASK_TCC\AgeGender\gender_deploy.prototxt"
genderModel = r"C:\Users\KinWah\MASK_TCC\AgeGender\gender_net.caffemodel"

MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
ageList = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
genderList = ['Male', 'Female']

crianca = ['(0-2)', '(4-6)', '(8-12)']
adolescente = ['(15-20)', '(25-32)']
Adulto = ['(38-43)', '(48-53)', '(60-100)']

# Load network
ageNet = cv.dnn.readNet(ageModel, ageProto)
genderNet = cv.dnn.readNet(genderModel, genderProto)
faceNet = cv.dnn.readNet(faceModel, faceProto)

# Open a video file or an image file or a camera stream
cap = cv.VideoCapture(0)
padding = 20
while cv.waitKey(1) < 0:
    # Read frame
    t = time.time()
    hasFrame, frame = cap.read()
    if not hasFrame:
        cv.waitKey()
        break

    frameFace, bboxes = getFaceBox(faceNet, frame)
    if not bboxes:
        print("No face Detected, Checking next frame")
        continue

    for bbox in bboxes:
        # print(bbox)
        face = frame[max(0,bbox[1]-padding):min(bbox[3]+padding,frame.shape[0]-1),max(0,bbox[0]-padding):min(bbox[2]+padding, frame.shape[1]-1)]

        blob = cv.dnn.blobFromImage(face, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
        genderNet.setInput(blob)
        genderPreds = genderNet.forward()
        gender = genderList[genderPreds[0].argmax()]
        # print("Gender Output : {}".format(genderPreds))
        print("Gender : {}, conf = {:.3f}".format(gender, genderPreds[0].max()))

        ageNet.setInput(blob)
        agePreds = ageNet.forward()
        age = ageList[agePreds[0].argmax()]
        
        print("Age Output : {}".format(agePreds))
        print("Age : {}, conf = {:.3f}".format(age, agePreds[0].max()))

        
        
        if age in crianca:
            age = 'Crianca'
        elif age in adolescente:
            age = 'Adolescente'
        else:
            age = 'Adulto'
        
        label = "{},{}".format(gender, age)
        cv.putText(frameFace, label, (bbox[0], bbox[1]-10), cv.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2, cv.LINE_AA)
        cv.imshow("Age Gender Demo", frameFace)
        # cv.imwrite("age-gender-out-{}".format(args.input),frameFace)
        
    print("time : {:.3f}".format(time.time() - t))
    key = cv.waitKey(1)
    if key == ord('q'):
        break

cap.release()
cv.destroyAllWindows()

Gender : Female, conf = 0.914
Age Output : [[5.4605704e-02 7.8825331e-01 1.5029566e-01 1.7881986e-03 3.8075112e-03
  5.8568286e-04 1.6631807e-04 4.9758883e-04]]
Age : (4-6), conf = 0.788
time : 0.435
Gender : Male, conf = 1.000
Age Output : [[2.1892085e-06 4.7180660e-05 8.5972089e-01 1.0745722e-01 3.2172959e-02
  5.8382622e-04 1.0324637e-05 5.3552885e-06]]
Age : (8-12), conf = 0.860
time : 0.118
Gender : Male, conf = 1.000
Age Output : [[1.1883207e-06 5.4274020e-05 9.0789664e-01 8.4454641e-02 7.3109358e-03
  2.7603499e-04 4.2691408e-06 2.1193036e-06]]
Age : (8-12), conf = 0.908
time : 0.110
Gender : Male, conf = 1.000
Age Output : [[1.1402415e-06 3.0006500e-05 8.6244726e-01 1.1381297e-01 2.3311483e-02
  3.8589732e-04 7.4865379e-06 3.6707818e-06]]
Age : (8-12), conf = 0.862
time : 0.113
Gender : Male, conf = 1.000
Age Output : [[1.9944282e-06 6.0881172e-05 8.7903064e-01 8.0360837e-02 4.0150501e-02
  3.7768451e-04 1.2406955e-05 5.1798670e-06]]
Age : (8-12), conf = 0.879
time : 0.094
Gend

Gender : Male, conf = 1.000
Age Output : [[8.06302251e-06 1.02475926e-04 7.52210855e-01 1.98150352e-01
  4.83991206e-02 1.08314282e-03 2.46608979e-05 2.13696640e-05]]
Age : (8-12), conf = 0.752
time : 0.134
Gender : Male, conf = 1.000
Age Output : [[1.22260035e-05 9.69062603e-05 5.89562654e-01 3.09744000e-01
  9.90244597e-02 1.49003381e-03 3.41618652e-05 3.55414923e-05]]
Age : (8-12), conf = 0.590
time : 0.123
Gender : Male, conf = 1.000
Age Output : [[1.3307117e-05 1.1127916e-04 4.3458733e-01 4.4226331e-01 1.2124036e-01
  1.6880962e-03 4.9291095e-05 4.7090394e-05]]
Age : (15-20), conf = 0.442
time : 0.132
Gender : Male, conf = 1.000
Age Output : [[5.0761291e-06 3.7778809e-05 1.3173492e-02 9.6527618e-01 2.0677513e-02
  7.9311832e-04 1.9031084e-05 1.7979166e-05]]
Age : (15-20), conf = 0.965
time : 0.124
Gender : Male, conf = 1.000
Age Output : [[4.4573076e-06 3.5879009e-05 6.9755348e-03 9.7970772e-01 1.2745162e-02
  5.0213409e-04 1.6755626e-05 1.2519129e-05]]
Age : (15-20), conf = 0.980

  7.42700174e-02 1.46111764e-03 4.15218783e-05 4.43846075e-05]]
Age : (15-20), conf = 0.914
time : 0.124
Gender : Male, conf = 1.000
Age Output : [[1.4514533e-05 9.8059245e-05 2.2332352e-02 9.0351588e-01 7.2562300e-02
  1.3755335e-03 4.9650331e-05 5.1699022e-05]]
Age : (15-20), conf = 0.904
time : 0.139
Gender : Male, conf = 1.000
Age Output : [[6.9195257e-06 3.8282218e-05 5.2459007e-03 9.4591212e-01 4.7746602e-02
  9.8946085e-04 2.8233966e-05 3.2442469e-05]]
Age : (15-20), conf = 0.946
time : 0.132
Gender : Male, conf = 1.000
Age Output : [[3.3276840e-06 2.2663067e-05 1.0220673e-02 9.6431780e-01 2.4765396e-02
  6.4072397e-04 1.4809856e-05 1.4641322e-05]]
Age : (15-20), conf = 0.964
time : 0.127
Gender : Male, conf = 1.000
Age Output : [[6.7472970e-06 4.5420573e-05 2.7932415e-02 9.0021473e-01 7.0873953e-02
  8.6053519e-04 3.5692490e-05 3.0328298e-05]]
Age : (15-20), conf = 0.900
time : 0.131
Gender : Male, conf = 1.000
Age Output : [[8.70389886e-06 6.03613407e-05 1.29732555e-02 9.51384

# INPUT PHOTO CRIANCA - ADOLESCENTE - ADULTO 

In [ ]:
# Import required modules
import cv2 as cv
import math
import time
import argparse

def getFaceBox(net, frame, conf_threshold=0.7):
    frameOpencvDnn = frame.copy()
    frameHeight = frameOpencvDnn.shape[0]
    frameWidth = frameOpencvDnn.shape[1]
    blob = cv.dnn.blobFromImage(frameOpencvDnn, 1.0, (300, 300), [104, 117, 123], True, False)

    net.setInput(blob)
    detections = net.forward()
    bboxes = []
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > conf_threshold:
            x1 = int(detections[0, 0, i, 3] * frameWidth)
            y1 = int(detections[0, 0, i, 4] * frameHeight)
            x2 = int(detections[0, 0, i, 5] * frameWidth)
            y2 = int(detections[0, 0, i, 6] * frameHeight)
            bboxes.append([x1, y1, x2, y2])
            cv.rectangle(frameOpencvDnn, (x1, y1), (x2, y2), (0, 255, 0), int(round(frameHeight/150)), 8)
    return frameOpencvDnn, bboxes



faceProto = r"C:\Users\KinWah\MASK_TCC\AgeGender\opencv_face_detector.pbtxt"
faceModel = r"C:\Users\KinWah\MASK_TCC\AgeGender\opencv_face_detector_uint8.pb"

ageProto = r"C:\Users\KinWah\MASK_TCC\AgeGender\age_deploy.prototxt"
ageModel = r"C:\Users\KinWah\MASK_TCC\AgeGender\age_net.caffemodel"

genderProto = r"C:\Users\KinWah\MASK_TCC\AgeGender\gender_deploy.prototxt"
genderModel = r"C:\Users\KinWah\MASK_TCC\AgeGender\gender_net.caffemodel"

MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
ageList = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
genderList = ['Male', 'Female']

crianca = ['(0-2)', '(4-6)', '(8-12)']
adolescente = ['(15-20)', '(25-32)']
Adulto = ['(38-43)', '(48-53)', '(60-100)']

# Load network
ageNet = cv.dnn.readNet(ageModel, ageProto)
genderNet = cv.dnn.readNet(genderModel, genderProto)
faceNet = cv.dnn.readNet(faceModel, faceProto)

# Open a video file or an image file or a camera stream
cap = cv.VideoCapture(r"C:\Users\KinWah\MASK_TCC\AgeGender\images\dani.jpg")
padding = 20
while cv.waitKey(1) < 0:
    # Read frame
    t = time.time()
    hasFrame, frame = cap.read()
    if not hasFrame:
        cv.waitKey()
        break

    frameFace, bboxes = getFaceBox(faceNet, frame)
    if not bboxes:
        print("No face Detected, Checking next frame")
        continue

    for bbox in bboxes:
        # print(bbox)
        face = frame[max(0,bbox[1]-padding):min(bbox[3]+padding,frame.shape[0]-1),max(0,bbox[0]-padding):min(bbox[2]+padding, frame.shape[1]-1)]

        blob = cv.dnn.blobFromImage(face, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
        genderNet.setInput(blob)
        genderPreds = genderNet.forward()
        gender = genderList[genderPreds[0].argmax()]
        # print("Gender Output : {}".format(genderPreds))
        print("Gender : {}, conf = {:.3f}".format(gender, genderPreds[0].max()))

        ageNet.setInput(blob)
        agePreds = ageNet.forward()
        age = ageList[agePreds[0].argmax()]
        
        print("Age Output : {}".format(agePreds))
        print("Age : {}, conf = {:.3f}".format(age, agePreds[0].max()))

        
        
        if age in crianca:
            age = 'Crianca'
        elif age in adolescente:
            age = 'Adolescente'
        else:
            age = 'Adulto'
        
        label = "{},{}".format(gender, age)
        cv.putText(frameFace, label, (bbox[0], bbox[1]-10), cv.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2, cv.LINE_AA)
        cv.imshow("Age Gender Demo", frameFace)
        # cv.imwrite("age-gender-out-{}".format(args.input),frameFace)
        
    print("time : {:.3f}".format(time.time() - t))
    key = cv.waitKey(1)
    if key == ord('q'):
        break

cap.release()
cv.destroyAllWindows()# CRIANCA - ADOLESCENTE - ADULTO

Gender : Male, conf = 1.000
Age Output : [[4.6253940e-03 5.0112758e-02 1.2444453e-02 1.9694006e-03 9.0939796e-01
  1.5151523e-02 5.5914447e-03 7.0704764e-04]]
Age : (25-32), conf = 0.909
time : 0.197


In [ ]:
from __future__ import absolute_import

from __future__ import division

from __future__ import print_function



from datetime import datetime

import time

import os

import numpy as np

import tensorflow as tf

from data import distorted_inputs

import re

from tensorflow.contrib.layers import *



from tensorflow.contrib.slim.python.slim.nets.inception_v3 import inception_v3_base





TOWER_NAME = 'tower'



def select_model(name):

    if name.startswith('inception'):

        print('selected (fine-tuning) inception model')

        return inception_v3

    elif name == 'bn':

        print('selected batch norm model')

        return levi_hassner_bn

    print('selected default model')

    return levi_hassner





def get_checkpoint(checkpoint_path, requested_step=None, basename='checkpoint'):

    if requested_step is not None:



        model_checkpoint_path = '%s/%s-%s' % (checkpoint_path, basename, requested_step)

        if os.path.exists(model_checkpoint_path) is None:

            print('No checkpoint file found at [%s]' % checkpoint_path)

            exit(-1)

            print(model_checkpoint_path)

        print(model_checkpoint_path)

        return model_checkpoint_path, requested_step



    ckpt = tf.train.get_checkpoint_state(checkpoint_path)

    if ckpt and ckpt.model_checkpoint_path:

        # Restore checkpoint as described in top of this program

        print(ckpt.model_checkpoint_path)

        global_step = ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]



        return ckpt.model_checkpoint_path, global_step

    else:

        print('No checkpoint file found at [%s]' % checkpoint_path)

        exit(-1)



def _activation_summary(x):

    tensor_name = re.sub('%s_[0-9]*/' % TOWER_NAME, '', x.op.name)

    tf.summary.histogram(tensor_name + '/activations', x)

    tf.summary.scalar(tensor_name + '/sparsity', tf.nn.zero_fraction(x))



def inception_v3(nlabels, images, pkeep, is_training):



    batch_norm_params = {

        "is_training": is_training,

        "trainable": True,

        # Decay for the moving averages.

        "decay": 0.9997,

        # Epsilon to prevent 0s in variance.

        "epsilon": 0.001,

        # Collection containing the moving mean and moving variance.

        "variables_collections": {

            "beta": None,

            "gamma": None,

            "moving_mean": ["moving_vars"],

            "moving_variance": ["moving_vars"],

        }

    }

    weight_decay = 0.00004

    stddev=0.1

    weights_regularizer = tf.contrib.layers.l2_regularizer(weight_decay)

    with tf.variable_scope("InceptionV3", "InceptionV3", [images]) as scope:



        with tf.contrib.slim.arg_scope(

                [tf.contrib.slim.conv2d, tf.contrib.slim.fully_connected],

                weights_regularizer=weights_regularizer,

                trainable=True):

            with tf.contrib.slim.arg_scope(

                    [tf.contrib.slim.conv2d],

                    weights_initializer=tf.truncated_normal_initializer(stddev=stddev),

                    activation_fn=tf.nn.relu,

                    normalizer_fn=batch_norm,

                    normalizer_params=batch_norm_params):

                net, end_points = inception_v3_base(images, scope=scope)

                with tf.variable_scope("logits"):

                    shape = net.get_shape()

                    net = avg_pool2d(net, shape[1:3], padding="VALID", scope="pool")

                    net = tf.nn.dropout(net, pkeep, name='droplast')

                    net = flatten(net, scope="flatten")

    

    with tf.variable_scope('output') as scope:

        

        weights = tf.Variable(tf.truncated_normal([2048, nlabels], mean=0.0, stddev=0.01), name='weights')

        biases = tf.Variable(tf.constant(0.0, shape=[nlabels], dtype=tf.float32), name='biases')

        output = tf.add(tf.matmul(net, weights), biases, name=scope.name)

        _activation_summary(output)

    return output



def levi_hassner_bn(nlabels, images, pkeep, is_training):



    batch_norm_params = {

        "is_training": is_training,

        "trainable": True,

        # Decay for the moving averages.

        "decay": 0.9997,

        # Epsilon to prevent 0s in variance.

        "epsilon": 0.001,

        # Collection containing the moving mean and moving variance.

        "variables_collections": {

            "beta": None,

            "gamma": None,

            "moving_mean": ["moving_vars"],

            "moving_variance": ["moving_vars"],

        }

    }

    weight_decay = 0.0005

    weights_regularizer = tf.contrib.layers.l2_regularizer(weight_decay)



    with tf.variable_scope("LeviHassnerBN", "LeviHassnerBN", [images]) as scope:



        with tf.contrib.slim.arg_scope(

                [convolution2d, fully_connected],

                weights_regularizer=weights_regularizer,

                biases_initializer=tf.constant_initializer(1.),

                weights_initializer=tf.random_normal_initializer(stddev=0.005),

                trainable=True):

            with tf.contrib.slim.arg_scope(

                    [convolution2d],

                    weights_initializer=tf.random_normal_initializer(stddev=0.01),

                    normalizer_fn=batch_norm,

                    normalizer_params=batch_norm_params):



                conv1 = convolution2d(images, 96, [7,7], [4, 4], padding='VALID', biases_initializer=tf.constant_initializer(0.), scope='conv1')

                pool1 = max_pool2d(conv1, 3, 2, padding='VALID', scope='pool1')

                conv2 = convolution2d(pool1, 256, [5, 5], [1, 1], padding='SAME', scope='conv2') 

                pool2 = max_pool2d(conv2, 3, 2, padding='VALID', scope='pool2')

                conv3 = convolution2d(pool2, 384, [3, 3], [1, 1], padding='SAME', biases_initializer=tf.constant_initializer(0.), scope='conv3')

                pool3 = max_pool2d(conv3, 3, 2, padding='VALID', scope='pool3')

                # can use tf.contrib.layer.flatten

                flat = tf.reshape(pool3, [-1, 384*6*6], name='reshape')

                full1 = fully_connected(flat, 512, scope='full1')

                drop1 = tf.nn.dropout(full1, pkeep, name='drop1')

                full2 = fully_connected(drop1, 512, scope='full2')

                drop2 = tf.nn.dropout(full2, pkeep, name='drop2')



    with tf.variable_scope('output') as scope:

        

        weights = tf.Variable(tf.random_normal([512, nlabels], mean=0.0, stddev=0.01), name='weights')

        biases = tf.Variable(tf.constant(0.0, shape=[nlabels], dtype=tf.float32), name='biases')

        output = tf.add(tf.matmul(drop2, weights), biases, name=scope.name)



    return output



def levi_hassner(nlabels, images, pkeep, is_training):



    weight_decay = 0.0005

    weights_regularizer = tf.contrib.layers.l2_regularizer(weight_decay)

    with tf.variable_scope("LeviHassner", "LeviHassner", [images]) as scope:



        with tf.contrib.slim.arg_scope(

                [convolution2d, fully_connected],

                weights_regularizer=weights_regularizer,

                biases_initializer=tf.constant_initializer(1.),

                weights_initializer=tf.random_normal_initializer(stddev=0.005),

                trainable=True):

            with tf.contrib.slim.arg_scope(

                    [convolution2d],

                    weights_initializer=tf.random_normal_initializer(stddev=0.01)):



                conv1 = convolution2d(images, 96, [7,7], [4, 4], padding='VALID', biases_initializer=tf.constant_initializer(0.), scope='conv1')

                pool1 = max_pool2d(conv1, 3, 2, padding='VALID', scope='pool1')

                norm1 = tf.nn.local_response_normalization(pool1, 5, alpha=0.0001, beta=0.75, name='norm1')

                conv2 = convolution2d(norm1, 256, [5, 5], [1, 1], padding='SAME', scope='conv2') 

                pool2 = max_pool2d(conv2, 3, 2, padding='VALID', scope='pool2')

                norm2 = tf.nn.local_response_normalization(pool2, 5, alpha=0.0001, beta=0.75, name='norm2')

                conv3 = convolution2d(norm2, 384, [3, 3], [1, 1], biases_initializer=tf.constant_initializer(0.), padding='SAME', scope='conv3')

                pool3 = max_pool2d(conv3, 3, 2, padding='VALID', scope='pool3')

                flat = tf.reshape(pool3, [-1, 384*6*6], name='reshape')

                full1 = fully_connected(flat, 512, scope='full1')

                drop1 = tf.nn.dropout(full1, pkeep, name='drop1')

                full2 = fully_connected(drop1, 512, scope='full2')

                drop2 = tf.nn.dropout(full2, pkeep, name='drop2')



    with tf.variable_scope('output') as scope:

        

        weights = tf.Variable(tf.random_normal([512, nlabels], mean=0.0, stddev=0.01), name='weights')

        biases = tf.Variable(tf.constant(0.0, shape=[nlabels], dtype=tf.float32), name='biases')

        output = tf.add(tf.matmul(drop2, weights), biases, name=scope.name)

    return output

ModuleNotFoundError: No module named 'data'